# Modern NLP on OpenFDA dataset

#### Credits:
(http://pydata.org/dc2016/schedule/presentation/11/). To view the video of the presentation on YouTube, see [here](https://www.youtube.com/watch?v=6zm9NC9uRkk)._

## Our Trail Map
An end-to-end data science & natural language processing pipeline, starting with **raw data** and running through **preparing**, **modeling**, **visualizing**, and **analyzing** the data. We'll touch on the following points:
1. A tour of the dataset
1. Introduction to text processing with spaCy
1. Automatic phrase modeling
1. Topic modeling with LDA
1. Visualizing topic models with pyLDAvis
1. Word vector models with word2vec
1. Visualizing word2vec with t-SNE

## The OpenFDA Dataset

In [1]:
import os
import codecs

data_directory = os.path.join('openfda_data')

businesses_filepath = os.path.join(data_directory,
                                   'drug-enforcement-0001-of-0001.json')

In [2]:
import json

recall_reasons = set()

with open(businesses_filepath) as data_file:    
    data = json.load(data_file)
    
    # iterate through each line (json record) in the file
    for business_json in data["results"]:                  
        recall_reasons.add(business_json["reason_for_recall"])

# turn restaurant_ids into a frozenset, as we don't need to change it anymore
recall_reasons_uniq = frozenset(recall_reasons)

# print the number of unique restaurant ids in the dataset
print("{:,}".format(len(recall_reasons_uniq)), "Recall Reasons in the dataset.")

2,067 Recall Reasons in the dataset.


Next, we will create a new file that contains only the text from recall reasons about drugs, with one reason per line in the file.

In [3]:
intermediate_directory = os.path.join('d:', 'open_fda_intermediate')

recall_txt_filepath = os.path.join(intermediate_directory,
                                   'recall_reasons_all.txt')

In [4]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 0 == 1:
    
    reason_count = 0

    # create & open a new file in write mode
    with codecs.open(recall_txt_filepath, 'w', encoding='utf_8') as recall_txt_file:

        # open the existing review json file
        with open(businesses_filepath) as data_file:
            data = json.load(data_file)

            for drug_details in data["results"]:
                all_reasons = drug_details["reason_for_recall"].split(';')
                
                # if this recall reason is empty or less than 5 characters dont add to final file
                for reason in all_reasons:
                    if reason is None or reason is " " or len(reason) <= 10:
                        continue

                    # write the restaurant review as a line in the new file
                    # escape newline characters in the original review text
                    recall_txt_file.write(reason.replace('\n', '\\n') + '\n')
                    reason_count += 1

    print("Text from {:,} recall reasons written to the new txt file.".format(reason_count))
    
else:
    
    with codecs.open(recall_txt_filepath, encoding='utf_8') as recall_txt_file:
        for reason_count, line in enumerate(recall_txt_file):
            pass
        
    print("Text from {:,} recall reasons in the txt file.".format(reason_count + 1))

Text from 8,931 recall reasons in the txt file.
Wall time: 280 ms


## spaCy &mdash; Industrial-Strength NLP in Python

![spaCy](https://s3.amazonaws.com/skipgram-images/spaCy.png)

[**spaCy**](https://spacy.io) is an industrial-strength natural language processing (_NLP_) library for Python. spaCy's goal is to take recent advancements in natural language processing out of research papers and put them in the hands of users to build production software.

spaCy handles many tasks commonly associated with building an end-to-end natural language processing pipeline:
- Tokenization
- Text normalization, such as lowercasing, stemming/lemmatization
- Part-of-speech tagging
- Syntactic dependency parsing
- Sentence boundary detection
- Named entity recognition and annotation

In the "batteries included" Python tradition, spaCy contains built-in data and models which you can use out-of-the-box for processing general-purpose English language text:
- Large English vocabulary, including stopword lists
- Token "probabilities"
- Word vectors

spaCy is written in optimized Cython, which means it's _fast_. According to a few independent sources, it's the fastest syntactic parser available in any language. Key pieces of the spaCy parsing pipeline are written in pure C, enabling efficient multithreading (i.e., spaCy can release the _GIL_).

In [5]:
import spacy
import pandas as pd
import itertools as it

nlp = spacy.load('en')

Let's grab a sample review to play with.

In [6]:
with codecs.open(recall_txt_filepath, encoding='utf_8') as f:
    sample_reason = list(it.islice(f, 8, 9))[0]
    sample_reason = sample_reason.replace('\\n', '\n')
        
print(sample_reason)

Lack of Assurance of Sterility: Franck's Lab Inc. initiated a recall of all Sterile Human Drugs distributed between 11/21/2011 and 05/21/2012 because FDA environmental sampling revealed the presence of microorganisms and fungal growth in the clean room where sterile products were prepared. 



Hand the review text to spaCy, and be prepared to wait...

In [7]:
%%time
parsed_reason = nlp(sample_reason)

Wall time: 12 ms


In [8]:
print(parsed_reason)

Lack of Assurance of Sterility: Franck's Lab Inc. initiated a recall of all Sterile Human Drugs distributed between 11/21/2011 and 05/21/2012 because FDA environmental sampling revealed the presence of microorganisms and fungal growth in the clean room where sterile products were prepared. 



Looks the same! What happened under the hood?

What about sentence detection and segmentation?

In [9]:
for num, sentence in enumerate(parsed_reason.sents):
    print('Sentence {}:'.format(num + 1))
    print(sentence)
    print('')

Sentence 1:
Lack of Assurance of Sterility: Franck's Lab Inc. initiated a recall of all Sterile Human Drugs distributed between 11/21/2011 and 05/21/2012 because FDA environmental sampling revealed the presence of microorganisms and fungal growth in the clean room where sterile products were prepared. 




What about named entity detection?

In [10]:
for num, entity in enumerate(parsed_reason.ents):
    print('Entity {}:'.format(num + 1), entity, '-', entity.label_)
    print('')

Entity 1: FDA - ORG



What about part of speech tagging?

In [11]:
token_text = [token.orth_ for token in parsed_reason]
token_pos = [token.pos_ for token in parsed_reason]

pd.DataFrame(list(zip(token_text, token_pos)),
             columns=['token_text', 'part_of_speech'])

,token_text,part_of_speech
0,Lack,NOUN
1,of,ADP
2,Assurance,PROPN
3,of,ADP
4,Sterility,NOUN
5,:,PUNCT
6,Franck,PROPN
7,'s,PART
8,Lab,PROPN
9,Inc.,PROPN


What about text normalization, like stemming/lemmatization and shape analysis?

In [12]:
token_lemma = [token.lemma_ for token in parsed_reason]
token_shape = [token.shape_ for token in parsed_reason]

pd.DataFrame(list(zip(token_text, token_lemma, token_shape)),
             columns=['token_text', 'token_lemma', 'token_shape'])

,token_text,token_lemma,token_shape
0,Lack,lack,Xxxx
1,of,of,xx
2,Assurance,assurance,Xxxxx
3,of,of,xx
4,Sterility,sterility,Xxxxx
5,:,:,:
6,Franck,franck,Xxxxx
7,'s,'s,'x
8,Lab,lab,Xxx
9,Inc.,inc.,Xxx.


What about token-level entity analysis?

In [13]:
token_entity_type = [token.ent_type_ for token in parsed_reason]
token_entity_iob = [token.ent_iob_ for token in parsed_reason]

pd.DataFrame(list(zip(token_text, token_entity_type, token_entity_iob)),
             columns=['token_text', 'entity_type', 'inside_outside_begin'])

,token_text,entity_type,inside_outside_begin
0,Lack,,O
1,of,,O
2,Assurance,,O
3,of,,O
4,Sterility,,O
5,:,,O
6,Franck,,O
7,'s,,O
8,Lab,,O
9,Inc.,,O


What about a variety of other token-level attributes, such as the relative frequency of tokens, and whether or not a token matches any of these categories?
- stopword
- punctuation
- whitespace
- represents a number
- whether or not the token is included in spaCy's default vocabulary?

In [15]:
token_attributes = [(token.orth_,
                     token.prob,
                     token.is_stop,
                     token.is_punct,
                     token.is_space,
                     token.like_num,
                     token.is_oov)
                    for token in parsed_reason]

df = pd.DataFrame(token_attributes,
                  columns=['text',
                           'log_probability',
                           'stop?',
                           'punctuation?',
                           'whitespace?',
                           'number?',
                           'out of vocab.?'])

df.loc[:, 'stop?':'out of vocab.?'] = (df.loc[:, 'stop?':'out of vocab.?']
                                       .applymap(lambda x: 'Yes' if x else ''))
                                               
df

,text,log_probability,stop?,punctuation?,whitespace?,number?,out of vocab.?
0,Lack,-12.744525,,,,,
1,of,-4.275874,Yes,,,,
2,Assurance,-15.678530,,,,,
3,of,-4.275874,Yes,,,,
4,Sterility,-18.106260,,,,,
5,:,-6.128876,,Yes,,,
6,Franck,-16.658981,,,,,
7,'s,-4.830559,,,,,
8,Lab,-13.243361,,,,,
9,Inc.,-13.148737,,,,,


If the text you'd like to process is general-purpose English language text (i.e., not domain-specific, like medical literature), spaCy is ready to use out-of-the-box.

I think it will eventually become a core part of the Python data science ecosystem &mdash; it will do for natural language computing what other great libraries have done for numerical computing.

## Phrase Modeling

_Phrase modeling_ is another approach to learning combinations of tokens that together represent meaningful multi-word concepts. We can develop phrase models by looping over the the words in our reviews and looking for words that _co-occur_ (i.e., appear one after another) together much more frequently than you would expect them to by random chance. The formula our phrase models will use to determine whether two tokens $A$ and $B$ constitute a phrase is:

$$\frac{count(A\ B) - count_{min}}{count(A) * count(B)} * N > threshold$$

...where:
* $count(A)$ is the number of times token $A$ appears in the corpus
* $count(B)$ is the number of times token $B$ appears in the corpus
* $count(A\ B)$ is the number of times the tokens $A\ B$ appear in the corpus *in order*
* $N$ is the total size of the corpus vocabulary
* $count_{min}$ is a user-defined parameter to ensure that accepted phrases occur a minimum number of times
* $threshold$ is a user-defined parameter to control how strong of a relationship between two tokens the model requires before accepting them as a phrase

Once our phrase model has been trained on our corpus, we can apply it to new text. When our model encounters two tokens in new text that identifies as a phrase, it will merge the two into a single new token.

Phrase modeling is superficially similar to named entity detection in that you would expect named entities to become phrases in the model (so _new york_ would become *new\_york*). But you would also expect multi-word expressions that represent common concepts, but aren't specifically named entities (such as _happy hour_) to also become phrases in the model.

We turn to the indispensible [**gensim**](https://radimrehurek.com/gensim/index.html) library to help us with phrase modeling &mdash; the [**Phrases**](https://radimrehurek.com/gensim/models/phrases.html) class in particular.

In [16]:
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence

C:\Users\aashis_tiwari\AppData\Local\Continuum\Anaconda3\envs\tensorflow\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


As we're performing phrase modeling, we'll be doing some iterative data transformation at the same time. Our roadmap for data preparation includes:

1. Segment text of complete reviews into sentences & normalize text
1. First-order phrase modeling $\rightarrow$ _apply first-order phrase model to transform sentences_
1. Second-order phrase modeling $\rightarrow$ _apply second-order phrase model to transform sentences_
1. Apply text normalization and second-order phrase model to text of complete reviews

We'll use this transformed data as the input for some higher-level modeling approaches in the following sections.

First, let's define a few helper functions that we'll use for text normalization. In particular, the `lemmatized_sentence_corpus` generator function will use spaCy to:
- Iterate over the 1M reviews in the `review_txt_all.txt` we created before
- Segment the reviews into individual sentences
- Remove punctuation and excess whitespace
- Lemmatize the text

... and do so efficiently in parallel, thanks to spaCy's `nlp.pipe()` function.

In [17]:
def punct_space(token):
    """
    helper function to eliminate tokens
    that are pure punctuation or whitespace
    """
    
    return token.is_punct or token.is_space

def line_review(filename):
    """
    generator function to read in reviews from the file
    and un-escape the original line breaks in the text
    """
    
    with codecs.open(filename, encoding='utf_8') as f:
        for review in f:
            yield review.replace('\\n', '\n')
            
def lemmatized_sentence_corpus(filename):
    """
    generator function to use spaCy to parse reviews,
    lemmatize the text, and yield sentences
    """
    
    for parsed_reason in nlp.pipe(line_review(filename),
                                  batch_size=10000, n_threads=4):
        
        for sent in parsed_reason.sents:
            yield u' '.join([token.lemma_ for token in sent
                             if not punct_space(token)])

In [18]:
unigram_sentences_filepath = os.path.join(intermediate_directory,
                                          'unigram_sentences_all.txt')

Let's use the `lemmatized_sentence_corpus` generator to loop over the original review text, segmenting the reviews into individual sentences and normalizing the text. We'll write this data back out to a new file (`unigram_sentences_all`), with one normalized sentence per line. We'll use this data for learning our phrase models.

In [19]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 0 == 1:

    with codecs.open(unigram_sentences_filepath, 'w', encoding='utf_8') as f:
        for sentence in lemmatized_sentence_corpus(recall_txt_filepath):
            f.write(sentence + '\n')

Wall time: 0 ns


If your data is organized like our `unigram_sentences_all` file now is &mdash; a large text file with one document/sentence per line &mdash; gensim's [**LineSentence**](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.LineSentence) class provides a convenient iterator for working with other gensim components. It *streams* the documents/sentences from disk, so that you never have to hold the entire corpus in RAM at once. This allows you to scale your modeling pipeline up to potentially very large corpora.

In [20]:
unigram_sentences = LineSentence(unigram_sentences_filepath)

Let's take a look at a few sample sentences in our new, transformed file.

In [21]:
for unigram_sentence in it.islice(unigram_sentences, 23, 24):
    print(' '.join(unigram_sentence))
    print('')

fda environmental sampling reveal the presence of microorganism and fungal growth in the clean room where sterile product be prepared



Next, we'll learn a phrase model that will link individual words into two-word phrases. We'd expect words that together represent a specific concept, like "`ice cream`", to be linked together to form a new, single token: "`ice_cream`".

In [22]:
bigram_model_filepath = os.path.join(intermediate_directory, 'bigram_model_all')

In [23]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute modeling yourself.
if 0 == 1:

    bigram_model = Phrases(unigram_sentences)

    bigram_model.save(bigram_model_filepath)
    
# load the finished model from disk
bigram_model = Phrases.load(bigram_model_filepath)

Wall time: 254 ms


Now that we have a trained phrase model for word pairs, let's apply it to the review sentences data and explore the results.

In [24]:
bigram_sentences_filepath = os.path.join(intermediate_directory,
                                         'bigram_sentences_all.txt')

In [25]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 0 == 1:

    with codecs.open(bigram_sentences_filepath, 'w', encoding='utf_8') as f:
        
        for unigram_sentence in unigram_sentences:
            
            bigram_sentence = ' '.join(bigram_model[unigram_sentence])
            
            f.write(bigram_sentence + '\n')

Wall time: 0 ns


In [26]:
bigram_sentences = LineSentence(bigram_sentences_filepath)

In [27]:
for bigram_sentence in it.islice(bigram_sentences, 23, 24):
    print(' '.join(bigram_sentence))
    print('')

fda environmental_sampling reveal the presence of microorganism and fungal_growth in the clean_room where_sterile product be prepared



Looks like the phrase modeling worked! We now see two-word phrases, such as "`ice_cream`" and "`apple_pie`", linked together in the text as a single token. Next, we'll train a _second-order_ phrase model. We'll apply the second-order phrase model on top of the already-transformed data, so that incomplete word combinations like "`vanilla_ice cream`" will become fully joined to "`vanilla_ice_cream`". No disrespect intended to [Vanilla Ice](https://www.youtube.com/watch?v=rog8ou-ZepE), of course.

In [28]:
trigram_model_filepath = os.path.join(intermediate_directory,
                                      'trigram_model_all')

In [29]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute modeling yourself.
if 0 == 1:

    trigram_model = Phrases(bigram_sentences)

    trigram_model.save(trigram_model_filepath)
    
# load the finished model from disk
trigram_model = Phrases.load(trigram_model_filepath)

Wall time: 210 ms


We'll apply our trained second-order phrase model to our first-order transformed sentences, write the results out to a new file, and explore a few of the second-order transformed sentences.

In [30]:
trigram_sentences_filepath = os.path.join(intermediate_directory,
                                          'trigram_sentences_all.txt')

In [31]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 0 == 1:

    with codecs.open(trigram_sentences_filepath, 'w', encoding='utf_8') as f:
        
        for bigram_sentence in bigram_sentences:
            
            trigram_sentence = ' '.join(trigram_model[bigram_sentence])
            
            f.write(trigram_sentence + '\n')

Wall time: 0 ns


In [32]:
trigram_sentences = LineSentence(trigram_sentences_filepath)

In [33]:
for trigram_sentence in it.islice(trigram_sentences, 23, 24):
    print(' '.join(trigram_sentence))
    print('')

fda_environmental_sampling reveal the presence of microorganism and fungal_growth in the clean_room_where_sterile product be prepared



Looks like the second-order phrase model was successful. We're now seeing three-word phrases, such as "`vanilla_ice_cream`" and "`cinnamon_ice_cream`".

The final step of our text preparation process circles back to the complete text of the reviews. We're going to run the complete text of the reviews through a pipeline that applies our text normalization and phrase models.

In addition, we'll remove stopwords at this point. _Stopwords_ are very common words, like _a_, _the_, _and_, and so on, that serve functional roles in natural language, but typically don't contribute to the overall meaning of text. Filtering stopwords is a common procedure that allows higher-level NLP modeling techniques to focus on the words that carry more semantic weight.

Finally, we'll write the transformed text out to a new file, with one review per line.

In [34]:
trigram_reviews_filepath = os.path.join(intermediate_directory,
                                        'trigram_transformed_reviews_all.txt')

In [35]:
 %%time
# %debug

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 0 == 1:

    with codecs.open(trigram_reviews_filepath, 'w', encoding='utf_8') as f:
        
        for parsed_reason in nlp.pipe(line_review(recall_txt_filepath),
                                      batch_size=10000, n_threads=4):
            
            # lemmatize the text, removing punctuation and whitespace
            unigram_review = [token.lemma_ for token in parsed_reason
                              if not punct_space(token)]
            
            # apply the first-order and second-order phrase models
            bigram_review = bigram_model[unigram_review]
            trigram_review = trigram_model[bigram_review]
    
            # remove any remaining stopwords
            trigram_review = [term for term in trigram_review
                              if term not in spacy.en.stop_words.STOP_WORDS]
            
            
            # write the transformed review as a line in the new file
            trigram_review = ' '.join(trigram_review)
            f.write(trigram_review + '\n')

Wall time: 0 ns


Let's preview the results. We'll grab one review from the file with the original, untransformed text, grab the same review from the file with the normalized and transformed text, and compare the two.

In [37]:
print('Original:' + '\n')

for review in it.islice(line_review(recall_txt_filepath), 11, 12):
    print(review)

print('----' + '\n')
print('Transformed:' + '\n')

with codecs.open(trigram_reviews_filepath, encoding='utf_8') as f:
    for review in it.islice(f, 11, 12):
        print(review)

Original:

Lack of Assurance of Sterility: Franck's Lab Inc. initiated a recall of all Sterile Human Drugs distributed between 11/21/2011 and 05/21/2012 because FDA environmental sampling revealed the presence of microorganisms and fungal growth in the clean room where sterile products were prepared. 

----

Transformed:

lack assurance sterility franck_'s_lab_inc. initiate recall all_sterile_human drugs_distribute_between_11/21/2011 05/21/2012_because_fda environmental_sampling_reveal presence microorganism fungal_growth clean_room_where_sterile product prepared



You can see that most of the grammatical structure has been scrubbed from the text &mdash; capitalization, articles/conjunctions, punctuation, spacing, etc. However, much of the general semantic *meaning* is still present. Also, multi-word concepts such as "`friday_night`" and "`above_average`" have been joined into single tokens, as expected. The review text is now ready for higher-level modeling. 

## Topic Modeling with Latent Dirichlet Allocation (_LDA_)

*Topic modeling* is family of techniques that can be used to describe and summarize the documents in a corpus according to a set of latent "topics". For this demo, we'll be using [*Latent Dirichlet Allocation*](http://www.jmlr.org/papers/volume3/blei03a/blei03a.pdf) or LDA, a popular approach to topic modeling.

In many conventional NLP applications, documents are represented a mixture of the individual tokens (words and phrases) they contain. In other words, a document is represented as a *vector* of token counts. There are two layers in this model &mdash; documents and tokens &mdash; and the size or dimensionality of the document vectors is the number of tokens in the corpus vocabulary. This approach has a number of disadvantages:
* Document vectors tend to be large (one dimension for each token $\Rightarrow$ lots of dimensions)
* They also tend to be very sparse. Any given document only contains a small fraction of all tokens in the vocabulary, so most values in the document's token vector are 0.
* The dimensions are fully indepedent from each other &mdash; there's no sense of connection between related tokens, such as _knife_ and _fork_.

LDA injects a third layer into this conceptual model. Documents are represented as a mixture of a pre-defined number of *topics*, and the *topics* are represented as a mixture of the individual tokens in the vocabulary. The number of topics is a model hyperparameter selected by the practitioner. LDA makes a prior assumption that the (document, topic) and (topic, token) mixtures follow [*Dirichlet*](https://en.wikipedia.org/wiki/Dirichlet_distribution) probability distributions. This assumption encourages documents to consist mostly of a handful of topics, and topics to consist mostly of a modest set of the tokens.

![LDA](https://s3.amazonaws.com/skipgram-images/LDA.png)

LDA is fully unsupervised. The topics are "discovered" automatically from the data by trying to maximize the likelihood of observing the documents in your corpus, given the modeling assumptions. They are expected to capture some latent structure and organization within the documents, and often have a meaningful human interpretation for people familiar with the subject material.

We'll again turn to gensim to assist with data preparation and modeling. In particular, gensim offers a high-performance parallelized implementation of LDA with its [**LdaMulticore**](https://radimrehurek.com/gensim/models/ldamulticore.html) class.

In [38]:
from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore

import pyLDAvis
import pyLDAvis.gensim
import warnings
import _pickle as pickle

The first step to creating an LDA model is to learn the full vocabulary of the corpus to be modeled. We'll use gensim's [**Dictionary**](https://radimrehurek.com/gensim/corpora/dictionary.html) class for this.

In [39]:
trigram_dictionary_filepath = os.path.join(intermediate_directory,
                                           'trigram_dict_all.dict')

In [40]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to learn the dictionary yourself.
if 0 == 1:

    trigram_reviews = LineSentence(trigram_reviews_filepath)

    # learn the dictionary by iterating over all of the reviews
    trigram_dictionary = Dictionary(trigram_reviews)
    
    # filter tokens that are very rare or too common from
    # the dictionary (filter_extremes) and reassign integer ids (compactify)
    trigram_dictionary.filter_extremes(no_below=10, no_above=0.4)
    trigram_dictionary.compactify()

    trigram_dictionary.save(trigram_dictionary_filepath)
    
# load the finished dictionary from disk
trigram_dictionary = Dictionary.load(trigram_dictionary_filepath)

Wall time: 104 ms


Like many NLP techniques, LDA uses a simplifying assumption known as the [*bag-of-words* model](https://en.wikipedia.org/wiki/Bag-of-words_model). In the bag-of-words model, a document is represented by the counts of distinct terms that occur within it. Additional information, such as word order, is discarded. 

Using the gensim Dictionary we learned to generate a bag-of-words representation for each review. The `trigram_bow_generator` function implements this. We'll save the resulting bag-of-words reviews as a matrix.

In the following code, "bag-of-words" is abbreviated as `bow`.

In [41]:
trigram_bow_filepath = os.path.join(intermediate_directory,
                                    'trigram_bow_corpus_all.mm')

In [42]:
def trigram_bow_generator(filepath):
    """
    generator function to read reviews from a file
    and yield a bag-of-words representation
    """
    
    for reason in LineSentence(filepath):
        yield trigram_dictionary.doc2bow(reason)

In [43]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to build the bag-of-words corpus yourself.
if 0 == 1:

    # generate bag-of-words representations for
    # all reviews and save them as a matrix
    MmCorpus.serialize(trigram_bow_filepath,
                       trigram_bow_generator(trigram_reviews_filepath))
    
# load the finished bag-of-words corpus from disk
trigram_bow_corpus = MmCorpus(trigram_bow_filepath)

Wall time: 80 ms


With the bag-of-words corpus, we're finally ready to learn our topic model from the reviews. We simply need to pass the bag-of-words matrix and Dictionary from our previous steps to `LdaMulticore` as inputs, along with the number of topics the model should learn. For this demo, we're asking for 50 topics.

In [44]:
lda_model_filepath = os.path.join(intermediate_directory, 'lda_model_all')

In [45]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to train the LDA model yourself.
if 0 == 1:

    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        
        # workers => sets the parallelism, and should be
        # set to your number of physical cores minus one
        lda = LdaMulticore(trigram_bow_corpus,
                           num_topics=5,
                           id2word=trigram_dictionary,
                           workers=1)
    
    lda.save(lda_model_filepath)
    
# load the finished LDA model from disk
lda = LdaMulticore.load(lda_model_filepath)

Wall time: 187 ms


Our topic model is now trained and ready to use! Since each topic is represented as a mixture of tokens, you can manually inspect which tokens have been grouped together into which topics to try to understand the patterns the model has discovered in the data.

In [46]:
def explore_topic(topic_number, topn=10):
    """
    accept a user-supplied topic number and
    print out a formatted list of the top terms
    """
        
    print('{:20} {}'.format(u'term', u'frequency') + '\n')

    for term, frequency in lda.show_topic(topic_number, topn=10):
        print('{:20} {:.3f}'.format(term, round(frequency, 3)))

In [47]:
explore_topic(topic_number=0)

term                 frequency

presence             0.048
potency              0.040
tablet               0.031
specification_result 0.025
product              0.023
particulate_matter   0.023
stability_data_do_not 0.020
drug_package         0.020
find                 0.019
support_expiry_potential_loss 0.018


In [48]:
topic_names = {0: 'potency',
               1: 'expiry',
               2: 'penicillin',
               3: 'tablets',
               4: 'market without approved'}

In [49]:
topic_names_filepath = os.path.join(intermediate_directory, 'topic_names.pkl')

with open(topic_names_filepath, 'wb') as f:
    pickle.dump(topic_names, f)

In [50]:
LDAvis_data_filepath = os.path.join(intermediate_directory, 'ldavis_prepared')

In [51]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 0 == 1:

    LDAvis_prepared = pyLDAvis.gensim.prepare(lda, trigram_bow_corpus,
                                              trigram_dictionary)

    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
        
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, "rb") as f:
    LDAvis_prepared = pickle.load(f)

Wall time: 68 ms


`pyLDAvis.display(...)` displays the topic model visualization in-line in the notebook.

In [52]:
pyLDAvis.display(LDAvis_prepared)

### Describing text with LDA
Beyond data exploration, one of the key uses for an LDA model is providing a compact, quantitative description of natural language text. Once an LDA model has been trained, it can be used to represent free text as a mixture of the topics the model learned from the original corpus. This mixture can be interpreted as a probability distribution across the topics, so the LDA representation of a paragraph of text might look like 50% _Topic A_, 20% _Topic B_, 20% _Topic C_, and 10% _Topic D_.

To use an LDA model to generate a vector representation of new text, you'll need to apply any text preprocessing steps you used on the model's training corpus to the new text, too. For our model, the preprocessing steps we used include:
1. Using spaCy to remove punctuation and lemmatize the text
1. Applying our first-order phrase model to join word pairs
1. Applying our second-order phrase model to join longer phrases
1. Removing stopwords
1. Creating a bag-of-words representation

Once you've applied these preprocessing steps to the new text, it's ready to pass directly to the model to create an LDA representation. The `lda_description(...)` function will perform all these steps for us, including printing the resulting topical description of the input text.

In [53]:
def get_sample_recall(recall_number):
    """
    retrieve a particular review index
    from the reviews file and return it
    """
    
    return list(it.islice(line_review(recall_txt_filepath),
                          recall_number, recall_number+1))[0]

In [54]:
def lda_description(recall_text, min_topic_freq=0.05):
    """
    accept the original text of a review and (1) parse it with spaCy,
    (2) apply text pre-proccessing steps, (3) create a bag-of-words
    representation, (4) create an LDA representation, and
    (5) print a sorted list of the top topics in the LDA representation
    """
    
    # parse the review text with spaCy
    parsed_reason = nlp(recall_text)
    
    # lemmatize the text and remove punctuation and whitespace
    unigram_reason = [token.lemma_ for token in parsed_reason
                      if not punct_space(token)]
    
    # apply the first-order and secord-order phrase models
    bigram_reason = bigram_model[unigram_reason]
    trigram_reason = trigram_model[bigram_reason]
    
    # remove any remaining stopwords
    trigram_reason = [term for term in trigram_reason
                      if not term in spacy.en.stop_words.STOP_WORDS]
    
    # create a bag-of-words representation
    reason_bow = trigram_dictionary.doc2bow(trigram_reason)
    
    # create an LDA representation
    reason_lda = lda[reason_bow]
    
    # sort with the most highly related topics first
    reason_lda = sorted(reason_lda, key=lambda topic_lda: -topic_lda[1])
    
    for topic_number, freq in reason_lda:
        if freq < min_topic_freq:
            break
            
        # print the most highly related topic names and frequencies
        print('{:2} {}'.format(topic_names[topic_number],
                                round(freq, 2)))

In [55]:
sample_reason = get_sample_recall(25)
print(sample_reason)

Presence of Foreign Substance: Recall is being initiated due to the presence of a foreign particle identified from a customer complaint.



In [56]:
lda_description(sample_reason)

potency 0.79
expiry 0.15


C:\Users\aashis_tiwari\AppData\Local\Continuum\Anaconda3\envs\tensorflow\lib\site-packages\gensim\models\phrases.py:274: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [57]:
sample_reason = get_sample_recall(100)
print(sample_reason)

 Lack of Assurance of Sterility: Franck's Lab Inc. initiated a recall of all Sterile Human Drugs distributed between 11/21/2011 and 05/21/2012. FDA environmental sampling revealed the presence of microorganisms and fungal growth in the clean room where sterile products were prepared. 



In [58]:
lda_description(sample_reason)

expiry 0.95


C:\Users\aashis_tiwari\AppData\Local\Continuum\Anaconda3\envs\tensorflow\lib\site-packages\gensim\models\phrases.py:274: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


## Word Vector Embedding with Word2Vec

Pop quiz! Can you complete this text snippet?

<br><br>

![word2vec quiz](https://s3.amazonaws.com/skipgram-images/word2vec-1.png)

<br><br><br>
You just demonstrated the core machine learning concept behind word vector embedding models!
<br><br><br>

![word2vec quiz 2](https://s3.amazonaws.com/skipgram-images/word2vec-2.png)

The goal of *word vector embedding models*, or *word vector models* for short, is to learn dense, numerical vector representations for each term in a corpus vocabulary. If the model is successful, the vectors it learns about each term should encode some information about the *meaning* or *concept* the term represents, and the relationship between it and other terms in the vocabulary. Word vector models are also fully unsupervised &mdash they learn all of these meanings and relationships solely by analyzing the text of the corpus, without any advance knowledge provided.

Perhaps the best-known word vector model is [word2vec](https://arxiv.org/pdf/1301.3781v3.pdf), originally proposed in 2013. The general idea of word2vec is, for a given *focus word*, to use the *context* of the word &mdash; i.e., the other words immediately before and after it &mdash; to provide hints about what the focus word might mean. To do this, word2vec uses a *sliding window* technique, where it considers snippets of text only a few tokens long at a time.

At the start of the learning process, the model initializes random vectors for all terms in the corpus vocabulary. The model then slides the window across every snippet of text in the corpus, with each word taking turns as the focus word. Each time the model considers a new snippet, it tries to learn some information about the focus word based on the surrouding context, and it "nudges" the words' vector representations accordingly. One complete pass sliding the window across all of the corpus text is known as a training *epoch*. It's common to train a word2vec model for multiple passes/epochs over the corpus. Over time, the model rearranges the terms' vector representations such that terms that frequently appear in similar contexts have vector representations that are *close* to each other in vector space.

For a deeper dive into word2vec's machine learning process, see [here](https://arxiv.org/pdf/1411.2738v4.pdf).

Word2vec has a number of user-defined hyperparameters, including:
- The dimensionality of the vectors. Typical choices include a few dozen to several hundred.
- The width of the sliding window, in tokens. Five is a common default choice, but narrower and wider windows are possible.
- The number of training epochs.

For using word2vec in Python, [gensim](https://rare-technologies.com/deep-learning-with-word2vec-and-gensim/) comes to the rescue again! It offers a [highly-optimized](https://rare-technologies.com/word2vec-in-python-part-two-optimizing/), [parallelized](https://rare-technologies.com/parallelizing-word2vec-in-python/) implementation of the word2vec algorithm with its [Word2Vec](https://radimrehurek.com/gensim/models/word2vec.html) class.

In [59]:
from gensim.models import Word2Vec

trigram_sentences = LineSentence(trigram_sentences_filepath)
word2vec_filepath = os.path.join(intermediate_directory, 'word2vec_model_all')

We'll train our word2vec model using the normalized sentences with our phrase models applied. We'll use 100-dimensional vectors, and set up our training process to run for twelve epochs.

In [110]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to train the word2vec model yourself.
if 1 == 1:

    # initiate the model and perform the first epoch of training
    recall2vec = Word2Vec(trigram_sentences, size=100, window=5,
                        min_count=20, sg=1, workers=1)
    
    recall2vec.save(word2vec_filepath)

    # perform another 11 epochs of training
    for i in range(1,12):

        recall2vec.train(trigram_sentences)
        recall2vec.save(word2vec_filepath)
        
# load the finished model from disk
recall2vec = Word2Vec.load(word2vec_filepath)
recall2vec.init_sims()

print('{} training epochs so far.'.format(recall2vec.train_count))

12 training epochs so far.
Wall time: 17.8 s


In [111]:
print('{:,} terms in the recall2vec vocabulary.'.format(len(recall2vec.wv.vocab)))

422 terms in the recall2vec vocabulary.


Let's take a peek at the word vectors our model has learned. We'll create a pandas DataFrame with the terms as the row labels, and the 100 dimensions of the word vector model as the columns.

In [62]:
# build a list of the terms, integer indices,
# and term counts from the food2vec model vocabulary
ordered_vocab = [(term, voc.index, voc.count)
                 for term, voc in recall2vec.wv.vocab.items()]

# sort by the term counts, so the most common terms appear first
ordered_vocab = sorted(ordered_vocab, key=lambda term_index: -term_index[2])

# unzip the terms, integer indices, and counts into separate lists
ordered_terms, term_indices, term_counts = zip(*ordered_vocab)

# create a DataFrame with the food2vec vectors as data,
# and the terms as row labels
word_vectors = pd.DataFrame(recall2vec.wv.syn0[term_indices, :],
                            index=ordered_terms)

word_vectors

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
of,0.250408,-0.426319,-0.465205,-0.204381,-0.392508,2.501663e-01,-0.158044,0.304148,0.218495,0.383973,...,-0.232480,-0.333301,0.318591,0.029734,-0.192704,-0.228435,-0.041155,-0.158670,0.077506,-0.346080
be,0.349246,-0.090556,-0.084044,-0.466857,-0.324749,-5.094894e-02,-0.038947,0.237301,0.152482,0.056053,...,-0.071386,0.110380,-0.113990,0.206021,0.123648,-0.001053,-0.013274,0.230379,-0.141617,-0.203462
the,0.067424,0.080021,0.033621,0.182714,-0.021266,-5.362771e-01,0.272588,0.268369,-0.154391,0.496174,...,0.333724,-0.015216,0.251310,0.469194,0.128453,0.241228,-0.147339,-0.153858,0.165506,-0.316384
sterility,0.303488,-0.228153,-0.219394,-0.302188,-0.070613,4.507700e-01,-0.125470,-0.174656,0.334442,0.810358,...,0.372313,0.377984,-0.309888,-0.746572,-0.342844,-0.191247,0.163880,0.044130,0.070255,-1.068096
assurance,0.240375,-0.008034,-0.594212,0.349680,-0.603484,5.307044e-01,0.029388,0.626848,0.348065,0.077353,...,0.388538,0.002734,-0.239271,-0.082077,-0.047349,-0.296976,-0.091350,0.045781,0.393641,-0.406683
product,0.167806,-0.021859,0.185118,-0.030679,-0.188094,1.409636e-01,-0.239155,0.544782,0.352739,0.016653,...,0.025098,-0.186757,-0.083697,-0.093634,-0.106546,-0.198794,0.290512,0.423234,-0.167009,-0.739767
lack,0.666243,-0.235590,-0.383695,0.450461,-0.533766,6.104476e-01,-0.394387,0.135181,-0.221998,0.284320,...,0.096456,0.144373,0.413397,-0.051289,-0.165963,-0.059666,-0.216124,-0.388248,0.065185,-0.543164
and,0.013336,0.060998,-0.056626,0.209782,0.027815,2.330013e-02,-0.311109,0.436705,-0.339965,0.205782,...,0.457187,-0.042881,0.268263,-0.257201,0.311483,-0.102934,0.405758,0.062987,-0.083985,-0.631277
in,0.106915,0.433992,-0.022223,-0.229789,-0.483802,-2.567068e-01,-0.353319,-0.128223,-0.179633,0.151007,...,0.287440,-0.491665,0.040266,0.345051,0.493042,0.187467,0.664412,0.068200,-0.363377,-0.660456
to,-0.055777,-0.142672,-0.052781,-0.160961,-0.114285,9.478991e-02,0.197403,0.107659,0.009553,0.110412,...,0.030555,0.313960,0.226491,0.116636,0.144624,0.104488,0.230591,-0.263950,-0.015821,-0.146170


### So... what can we do with all these numbers?
The first thing we can use them for is to simply look up related words and phrases for a given term of interest.

In [63]:
def get_related_terms(token, topn=5):
    """
    look up the topn most similar terms to token
    and print them as a formatted list
    """

    for word, similarity in recall2vec.most_similar(positive=[token], topn=topn):

        print('{:20} {}'.format(word, round(similarity, 3)))

### Issues with Syringe?

In [65]:
get_related_terms('syringe')

support_expiry_potential_loss 0.68
store                0.636
drug_package         0.626
defective_delivery_system 0.516
potency              0.492


### What about tablets?

In [66]:
get_related_terms('tablets')

package_insert       0.467
strength             0.432
bottle               0.431
fail_stability       0.399
contaminate          0.399


### Since we are mining for Drug Recalls, what are the top factors?

In [67]:
get_related_terms('recall', topn=20)

initiate             0.427
poor_sterile         0.405
production_practice_result 0.404
due                  0.397
quality_control_procedure 0.355
by                   0.353
concerned_test_result_obtain 0.345
05/21/2012           0.341
laboratory           0.341
fda                  0.341
contain_undeclared   0.341
observation_associate 0.34
tadalafil            0.338
reliable             0.336
unapproved_drug      0.33
inc.                 0.326
laboratory_result    0.325
be                   0.325
not_expire_due       0.322
manufacturer         0.317


## Word algebra!
No self-respecting word2vec demo would be complete without a healthy dose of *word algebra*, also known as *analogy completion*.

The core idea is that once words are represented as numerical vectors, you can do math with them. The mathematical procedure goes like this:
1. Provide a set of words or phrases that you'd like to add or subtract.
1. Look up the vectors that represent those terms in the word vector model.
1. Add and subtract those vectors to produce a new, combined vector.
1. Look up the most similar vector(s) to this new, combined vector via cosine similarity.
1. Return the word(s) associated with the similar vector(s).

But more generally, you can think of the vectors that represent each word as encoding some information about the *meaning* or *concepts* of the word. What happens when you ask the model to combine the meaning and concepts of words in new ways? Let's see.

In [68]:
def word_algebra(add=[], subtract=[], topn=1):
    """
    combine the vectors associated with the words provided
    in add= and subtract=, look up the topn most similar
    terms to the combined vector, and print the result(s)
    """
    answers = recall2vec.most_similar(positive=add, negative=subtract, topn=topn)
    
    for term, similarity in answers:
        print(term)

### ?

In [109]:
word_algebra(add=['presence', 'calcium'])

stainless_steel


### ?

In [100]:
word_algebra(add=['sterility', 'syringe'], subtract=['label'])

support_expiry_recent


###  ?

In [108]:
word_algebra(add=['fda', 'product'], subtract=['sterile'])

sibutramine


## Word Vector Visualization with t-SNE

[t-Distributed Stochastic Neighbor Embedding](https://lvdmaaten.github.io/publications/papers/JMLR_2008.pdf), or *t-SNE* for short, is a dimensionality reduction technique to assist with visualizing high-dimensional datasets. It attempts to map high-dimensional data onto a low two- or three-dimensional representation such that the relative distances between points are preserved as closely as possible in both high-dimensional and low-dimensional space.

scikit-learn provides a convenient implementation of the t-SNE algorithm with its [TSNE](http://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html) class.

In [91]:
from sklearn.manifold import TSNE

Our input for t-SNE will be the DataFrame of word vectors we created before. Let's first:
1. Drop stopwords &mdash; it's probably not too interesting to visualize *the*, *of*, *or*, and so on
1. Take only the 5,000 most frequent terms in the vocabulary &mdash; no need to visualize all ~50,000 terms right now.

In [92]:
tsne_input = word_vectors.drop(spacy.en.stop_words.STOP_WORDS, errors='ignore')
tsne_input = tsne_input.head(1000)

In [93]:
tsne_input.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
sterility,0.303488,-0.228153,-0.219394,-0.302188,-0.070613,0.450770,-0.125470,-0.174656,0.334442,0.810358,...,0.372313,0.377984,-0.309888,-0.746572,-0.342844,-0.191247,0.163880,0.044130,0.070255,-1.068096
assurance,0.240375,-0.008034,-0.594212,0.349680,-0.603484,0.530704,0.029388,0.626848,0.348065,0.077353,...,0.388538,0.002734,-0.239271,-0.082077,-0.047349,-0.296976,-0.091350,0.045781,0.393641,-0.406683
product,0.167806,-0.021859,0.185118,-0.030679,-0.188094,0.140964,-0.239155,0.544782,0.352739,0.016653,...,0.025098,-0.186757,-0.083697,-0.093634,-0.106546,-0.198794,0.290512,0.423234,-0.167009,-0.739767
lack,0.666243,-0.235590,-0.383695,0.450461,-0.533766,0.610448,-0.394387,0.135181,-0.221998,0.284320,...,0.096456,0.144373,0.413397,-0.051289,-0.165963,-0.059666,-0.216124,-0.388248,0.065185,-0.543164
penicillin,0.331125,0.195782,-0.216822,-0.540945,-0.070391,-0.168526,0.119359,0.104435,-0.079008,0.072145,...,0.097845,0.443677,-0.943778,-0.739152,0.490247,-0.213078,0.071543,0.367104,-0.180235,-0.564265


In [94]:
tsne_filepath = os.path.join(intermediate_directory,
                             'tsne_model')

tsne_vectors_filepath = os.path.join(intermediate_directory,
                                     'tsne_vectors.npy')

In [95]:
%%time

if 0 == 1:
    
    tsne = TSNE()
    tsne_vectors = tsne.fit_transform(tsne_input.values)
    
    with open(tsne_filepath, 'wb') as f:
        pickle.dump(tsne, f)

    pd.np.save(tsne_vectors_filepath, tsne_vectors)
    
with open(tsne_filepath, "rb") as f:
    tsne = pickle.load(f)
    
tsne_vectors = pd.np.load(tsne_vectors_filepath)

tsne_vectors = pd.DataFrame(tsne_vectors,
                            index=pd.Index(tsne_input.index),
                            columns=['x_coord', 'y_coord'])

Wall time: 64 ms


Now we have a two-dimensional representation of our data! Let's take a look.

In [96]:
tsne_vectors.head()

,x_coord,y_coord
sterility,-8.427881,15.936145
assurance,7.898233,-1.578057
product,-20.207019,2.562714
lack,-26.316070,3.674049
penicillin,47.524378,11.912826


In [97]:
tsne_vectors['word'] = tsne_vectors.index

### Plotting with Bokeh

In [98]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, value

output_notebook()

C:\Users\aashis_tiwari\AppData\Local\Continuum\Anaconda3\envs\tensorflow\lib\site-packages\bokeh\core\json_encoder.py:52: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  NP_EPOCH = np.datetime64('1970-01-01T00:00:00Z')


Loading BokehJS ...

In [99]:
# add our DataFrame as a ColumnDataSource for Bokeh
plot_data = ColumnDataSource(tsne_vectors)

# create the plot and configure the
# title, dimensions, and tools
tsne_plot = figure(title='t-SNE Word Embeddings',
                   plot_width = 800,
                   plot_height = 800,
                   tools= ('pan, wheel_zoom, box_zoom,'
                           'box_select, resize, reset'),
                   active_scroll='wheel_zoom')

# add a hover tool to display words on roll-over
tsne_plot.add_tools( HoverTool(tooltips = '@word') )

# draw the words as circles on the plot
tsne_plot.circle('x_coord', 'y_coord', source=plot_data,
                 color='blue', line_alpha=0.2, fill_alpha=0.1,
                 size=10, hover_line_color='black')

# configure visual elements of the plot
tsne_plot.title.text_font_size = value('16pt')
tsne_plot.xaxis.visible = False
tsne_plot.yaxis.visible = False
tsne_plot.grid.grid_line_color = None
tsne_plot.outline_line_color = None

# engage!
show(tsne_plot);

## Conclusion

Whew! Let's round up the major components that we've seen:
1. Text processing with **spaCy**
1. Automated **phrase modeling**
1. Topic modeling with **LDA** $\ \longrightarrow\ $ visualization with **pyLDAvis**
1. Word vector modeling with **word2vec** $\ \longrightarrow\ $ visualization with **t-SNE**

#### Why use these models?
Dense vector representations for text like LDA and word2vec can greatly improve performance for a number of common, text-heavy problems like:
- Text classification
- Search
- Recommendations
- Question answering